In [179]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from dace_query.astrometry import Astrometry
from dace_query.spectroscopy import Spectroscopy
from kepmodel import astro, astrorv, rv
from spleaf import term
from copy import deepcopy

In [181]:
# import pandas as pd
# from dace_query.spectroscopy import Spectroscopy

# # Query CORALIE RV data
# print("Querying CORALIE RV data...")
# coralie_df = (
#     Spectroscopy.query_database(
#         filters={
#             "ins_name": {"contains": ["CORALIE98", "CORALIE07"]},
#             "spectro_ccf_rv_err": {"max": 50}
#         },
#         output_format="pandas"
#     )
#     .sort_values("obj_date_bjd")
#     .reset_index(drop=True)
# )

# # Display basic info and sample data
# print("\nCORALIE Dataframe Info:")
# print(coralie_df.info())
# print("\nFirst 5 rows of CORALIE Dataframe:")
# print(coralie_df.head())
# print("\nUnique star names in CORALIE data:")
# unique_stars = coralie_df["obj_id_catname"].unique()
# print(f"Found {len(unique_stars)} unique stars:")
# for i, star in enumerate(unique_stars[:10]):  # Show first 10
#     print(f"{i+1}. '{star}'")

coralie_df = pd.read_csv("complete_coralie_data.csv")



/var/folders/cb/dq7rbdjs1gbbgm7jgswln58m0000gn/T/ipykernel_46831/3807669149.py:29: DtypeWarning: Columns (4,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  coralie_df = pd.read_csv("complete_coralie_data.csv")


In [182]:
# import pandas as pd
# import os

# # Function to parse Hipparcos IAD files
# def parse_hipparcos_iad(filepath):
#     with open(filepath, 'r') as f:
#         lines = f.readlines()
    
#     hip_line = None
#     for i, line in enumerate(lines):
#         if line.strip().startswith('# HIP    MCE    NRES'):
#             for j in range(i + 1, len(lines)):
#                 if not lines[j].strip().startswith('#'):
#                     hip_line = lines[j]
#                     break
#             break
    
#     if hip_line is None:
#         raise ValueError(f"Could not find HIP ID in {filepath}")
    
#     hip_id = int(hip_line.split()[0].strip())
    
#     data_start = 0
#     for i, line in enumerate(lines):
#         if line.strip().startswith('# IORB   EPOCH    PARF'):
#             data_start = i + 1
#             break
    
#     if data_start == 0:
#         raise ValueError(f"Could not find data start in {filepath}")
    
#     data = []
#     for line in lines[data_start:]:
#         if not line.strip() or line.strip().startswith('#'):
#             continue
#         fields = line.split()
#         if len(fields) < 7:
#             continue
        
#         parf = float(fields[2])
#         cpsi = float(fields[3])
#         spsi = float(fields[4])
#         res = float(fields[5])
#         sres = float(fields[6])
        
#         # Calculate S_MAS: Absolute astrometric signal (RES + fitted model)
#         # This represents the total astrometric signal in mas (milliarcseconds)
#         # The fitted model component can be estimated from PARF (partial derivative)
#         model_component = abs(parf)
#         s_mas = abs(res) + model_component
        
#         # Calculate theta angle from PSI angle (following Gaia convention)
#         # In Hipparcos, PSI might be defined in the scanning reference frame
#         # In Gaia, theta is measured from North through East
#         # First, get the PSI angle
#         psi_angle = np.arctan2(spsi, cpsi)
        
#         # Convert to theta angle (following Gaia convention)
#         # This may involve a coordinate system transformation
#         # For simplicity, let's assume a π/2 rotation is needed (this would need verification)
#         theta_angle = psi_angle + np.pi/2
        
#         # Normalize theta to [0, 2π]
#         theta_angle = theta_angle % (2 * np.pi)
        
#         # Calculate CTH and STH from theta angle
#         cth = np.cos(theta_angle)
#         sth = np.sin(theta_angle)
        
#         data.append({
#             'HIP': hip_id,
#             'IORB': int(fields[0]),
#             'EPOCH': float(fields[1]),
#             'PARF': parf,
#             'CPSI': cpsi,
#             'SPSI': spsi,
#             'RES': res,
#             'SRES': sres,
#             'T_BJD': 2448349.0625 + (float(fields[1]) * 365.25),  # Convert to BJD
#             'S_MAS': s_mas,  # Absolute astrometric signal in mas
#             'CTH': cth,      # Cosine of theta angle (Gaia convention)
#             'STH': sth       # Sine of theta angle (Gaia convention)
#         })
    
#     return pd.DataFrame(data)

# # Load all Hipparcos data into a single dataframe
# def load_all_hipparcos_data(base_dir="ResRec_JavaTool_2014"):
#     print("Loading Hipparcos data...")
#     all_data = []
#     for folder in range(0, 121):  # H000 to H120
#         folder_name = f"H{folder:03d}"
#         folder_path = os.path.join(base_dir, folder_name)
#         if not os.path.exists(folder_path):
#             continue
#         for file_num in range(1, 1000):  # H000001.d to H000999.d
#             file_name = f"H{folder:03d}{file_num:03d}.d"
#             # print(file_name, end=', ')
#             file_path = os.path.join(folder_path, file_name)
#             if os.path.exists(file_path):
#                 try:
#                     df = parse_hipparcos_iad(file_path)
#                     all_data.append(df)
#                 except Exception as e:
#                     print(f"Error parsing {file_path}: {e}")
#     if not all_data:
#         raise ValueError("No Hipparcos data loaded")
#     return pd.concat(all_data, ignore_index=True)

# # Create Hipparcos dataframe
# hipparcos_df = load_all_hipparcos_data()

# # Display basic info and sample data
# print("\nHipparcos Dataframe Info:")
# print(hipparcos_df.info())
# print("\nFirst 5 rows of Hipparcos Dataframe:")
# print(hipparcos_df.head())
# print("\nUnique HIP IDs in Hipparcos data:")
# unique_hips = hipparcos_df["HIP"].unique()
# print(f"Found {len(unique_hips)} unique HIP IDs:")
# for i, hip in enumerate(unique_hips[:10]):  # Show first 10
#     print(f"{i+1}. {hip}")



In [183]:
# hipparcos_df

In [143]:
# from astroquery.simbad import Simbad

# # Function to query SIMBAD for basic data and identifiers
# def query_simbad(hd_name):
#     print(f"\nQuerying SIMBAD for: '{hd_name}'")
    
#     # Query basic object data
#     result = Simbad.query_object(hd_name)
#     print(result.columns)
#     if result is None or len(result) == 0:
#         print(f"No basic data found for '{hd_name}'")
#     else:
#         print("Basic Data:")
#         print(result[['main_id', 'ra', 'dec', 'coo_err_maj', 'coo_err_min', 'matched_id']])
    
#     # Query all identifiers
#     ids = Simbad.query_objectids(hd_name, criteria="ident.id LIKE 'HIP%'")
#     if ids is None or len(ids) == 0:
#         print(f"No identifiers found for '{hd_name}'")
#     else:
#         print("Identifiers:")
#         a = ids.to_pandas()
#         if len(a['id'].to_list()) > 0:
#             found_HIP = a['id'].to_list()[0]
#         print(found_HIP)

# # Test with a few HD IDs
# hd_ids = ["HD17289"]
# for hd_id in hd_ids:
#     query_simbad(hd_id)

In [206]:
from astroquery.simbad import Simbad

def get_hip_id(hd_name):
    result = Simbad.query_objectids(hd_name)
    if result:
        for id in result['id']:
            if 'HIP' in id:
                return int(id.split()[-1])
    return None

# Create HD->HIP mapping for all unique HD stars
# hd_hip_mapping = {hd: get_hip_id(hd) for hd in coralie_df['obj_id_catname'].unique()}
hd_hip_mapping = {hd: get_hip_id(hd) for hd in coralie_df['obj_id_catname'].unique() 
                  if hd.startswith('HD')}
print(f"Created mapping for {len(hd_hip_mapping)} HD stars")


Created mapping for 251 HD stars


In [207]:
print(hd_hip_mapping)

{'HD175518': 92918, 'HD13445': 10138, 'HD219709': 115087, 'HD1926': 1823, 'HD202457': 105214, 'HD3222': 2743, 'HD17289': 12726, 'HD10647': 7978, 'HD31143': 22602, 'HD20916': 15539, 'HD39091': 26394, 'HD40307': 27887, 'HD30501': 22122, 'HD32778': 23437, 'HD52698': 33817, 'HD74014': 42634, 'HD197214': 102264, 'HD1461': 1499, 'HD30562': 22336, 'HD82943': 47007, 'HD27019': 19758, 'HD62644': 37606, 'HD64606': 38625, 'HD46569': 31079, 'HD48189': 31711, 'HD50499': 32970, 'HD104304': 58576, 'HD29813': 21778, 'HD92987': 52472, 'HD112164': 63033, 'HD121384': 68101, 'HD87359': 49350, 'HD99610': 55900, 'HD62848': 37635, 'HD115153': 64688, 'HD145825': 79578, 'HD63077': 37853, 'HD78549': 44676, 'HD78643': 44874, 'HD78762': 44745, 'HD117635': 65982, 'HD136352': 75181, 'HD150248': 81746, 'HD152311': 82621, 'HD86249': 48718, 'HD98356': 55235, 'HD165499': 89042, 'HD92788': 52409, 'HD131923': 73241, 'HD134251': 74126, 'HD137812': 75762, 'HD138648': 76203, 'HD147584': 80686, 'HD153631': 83276, 'HD85390': 

In [208]:
# Filter CORALIE data with HIP matches
valid_stars = [hd for hd, hip in hd_hip_mapping.items() if hip is not None]
filtered_coralie = coralie_df[coralie_df['obj_id_catname'].isin(valid_stars)]

# Filter Hipparcos data for matched HIP IDs
# valid_hips = [hip for hip in hd_hip_mapping.values() if hip is not None]
# filtered_hipparcos = hipparcos_df[hipparcos_df['HIP'].isin(valid_hips)]

filtered_hipparcos = pd.read_csv("filtered_hipparcos.csv")




In [187]:
filtered_coralie.describe()

,Unnamed: 0,obj_date_bjd,spectro_analysis_ca_flx_h,spectro_analysis_ca_flx_r1,spectro_analysis_protn84_err,spectro_analysis_smw,spectro_analysis_ca_flx_k_err,spectro_analysis_ca_flx_r2_err,spectro_analysis_halpha_flx_r2_err,spectro_analysis_halpha_flx_h_err,...,ins1_adc2_temp,ins2_adc1_temp,ins2_adc2_temp,ins3_adc1_temp,ins3_adc2_temp,ins4_adc1_temp,ins4_adc2_temp,th_ar,th_ar1,th_ar2
count,4727.000000,4727.000000,4633.000000,4633.000000,0.0,4633.0,4633.000000,4633.000000,4633.000000,4633.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,3755.884282,53257.641694,-366.922481,-366.905116,NaN,-99999.0,-366.929035,-366.929094,-366.929204,-366.929131,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,3147.914727,1593.322594,6046.967322,6046.968376,NaN,0.0,6046.966924,6046.966920,6046.966914,6046.966918,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,4.000000,51126.518369,-99999.000000,-99999.000000,NaN,-99999.0,-99999.000000,-99999.000000,-99999.000000,-99999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1210.500000,51975.620500,0.001410,0.004588,NaN,-99999.0,0.000021,0.000017,0.000000,0.000018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2411.000000,52692.549470,0.004112,0.012315,NaN,-99999.0,0.000162,0.000098,0.000000,0.000071,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,6383.500000,54471.679643,0.008682,0.028930,NaN,-99999.0,0.000286,0.000182,0.000000,0.000109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,9985.000000,56953.572428,0.106230,0.452620,NaN,-99999.0,0.000713,0.000505,0.000000,0.000360,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
epoch_rjd = 50_000  # Reduced Julian Date epoch
epoch_bjd = 2_400_000  # Barycentric Julian Date epoch

# Defaults
default_hip_jitter = 0.0  # Default HIPPARCOS jitter value
# Default instrument jitter value
default_inst_jitter = {
    "CORALIE98": 5.0,
    "CORALIE07": 8.0,
}
# Default instrument colors for plotting
default_inst_color = {
    "CORALIE98": "deepskyblue",
    "CORALIE07": "darkviolet",
}

In [189]:
star_rv = filtered_coralie[filtered_coralie['obj_id_catname'] == 'HD17289']
star_astro = filtered_hipparcos[filtered_hipparcos['HIP'] == 'HIP 12726']

filtered_hipparcos.head()


,Unnamed: 0,HIP,IORB,EPOCH,PARF,CPSI,SPSI,RES,SRES,T_BJD,S_MAS,CTH,STH
0,4036262,305,305,-1.0353,0.6441,-0.3940,0.9191,-1.09,3.07,2.447971e+06,1.7341,-0.919109,-0.394004
1,4036263,305,362,-0.9660,-0.6689,0.7292,0.6843,-0.27,2.66,2.447996e+06,0.9389,-0.684300,0.729200
2,4036264,305,362,-0.9660,-0.6663,0.7267,0.6869,0.06,2.47,2.447996e+06,0.7263,-0.686926,0.726727
3,4036265,305,363,-0.9648,-0.6671,0.7251,0.6886,0.78,2.92,2.447997e+06,1.4471,-0.688621,0.725122
4,4036266,305,416,-0.9004,0.5023,-0.6959,0.7181,2.88,3.59,2.448020e+06,3.3823,-0.718120,-0.695919


In [190]:
# Rename hipparcos_df to astrometry_data for consistency with the tutorial
# astrometry_data = filtered_hipparcos

# Bin astrometric measurements by 1-day intervals to reduce noise
filtered_hipparcos["bin"] = (filtered_hipparcos["T_BJD"] // 1).astype(int)

# Compute weighted averages within each bin
w = 1 / filtered_hipparcos["SRES"] ** 2
astro_data = (
    filtered_hipparcos.groupby("bin")
    .apply(
        lambda x: pd.Series(
            {
                col: np.average(x[col], weights=w.loc[x.index])
                for col in x.columns
                if (col != "SRES") and (col != "IAD_SOURCE")
            }
            | {"SRES": np.sqrt(1 / np.sum(w.loc[x.index]))}
        )
    )
    .reset_index(drop=True)
)


/var/folders/cb/dq7rbdjs1gbbgm7jgswln58m0000gn/T/ipykernel_46831/3774836901.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


In [191]:
def initialize_models_per_star(hd_name, coralie_data, hipparcos_data, hd_hip_map):
    hip_id = hd_hip_map[hd_name]
    
    # Extract data for this star
    star_rv = coralie_data[coralie_data['obj_id_catname'] == hd_name]
    star_astro = hipparcos_data[hipparcos_data['HIP'] == hip_id]

    # 2.1 Astrometry Model Setup (From Tutorial Section 2)
    astro_model = astro.AstroModel(
        t=star_astro['T_BJD'].values,  # Time of observations
        s=star_astro['S_MAS'].values,  # Signal or measurement values
        cth=star_astro['CTH'].values,  # Cosine of true anomaly
        sth=star_astro['STH'].values,  # Sine of true anomaly
        epoch=star_astro['T_BJD'].values,
        parf=star_astro['PARF'].values,
        res=star_astro['RES'].values,
        sres=star_astro['SRES'].values,
        err=term.Error(star_astro["SRES"].values),
        jit=term.Jitter(0.0),  # Set default jitter # Will be optimized later
    )
    # astro_model = astro.AstroModel(
    #     star_astro["T_BJD"].values - epoch_bjd,
    #     star_astro["S_MAS"].values,
    #     star_astro["CTH"].values,
    #     star_astro["STH"].values,
    #     err=term.Error(star_astro["SRES"].values),
    #     jit=term.Jitter(default_hip_jitter),  # Set default jitter
    # )

    # 2.2 RV Model Initialization (From Tutorial Section 2)
    instruments = star_rv['ins_name'].unique()
    rv_model = rv.RVModel(
        t=star_rv['obj_date_bjd'].values,
        rv=star_rv['spectro_ccf_rv'].values,
        rv_err=star_rv['spectro_ccf_rv_err'].values,
        instruments=instruments,
        inst_jitters={inst: default_inst_jitter.get(inst, 5.0) for inst in instruments},
        inst_colors={inst: default_inst_color.get(inst, 'gray') for inst in instruments}
    )

    # 2.3 Joint Model Framework (From Tutorial Section 3)
    joint_model = astrorv.AstroRVModel(
        astrometry=astro_model,
        rv=rv_model,
        ms=1.01,  # Stellar mass - should be star-specific!
        epoch_rjd=50000,
        epoch_bjd=2400000
    )

    return {
        'hd_name': hd_name,
        'hip_id': hip_id,
        'astro_model': astro_model,
        'rv_model': rv_model,
        'joint_model': joint_model
    }


In [192]:
def initialize_all_models(hd_hip_mapping, coralie_df, hipparcos_df):
    models = {}
    for hd_name in tqdm(hd_hip_mapping.keys(), desc="Initializing models"):
        try:
            models[hd_name] = initialize_models_per_star(
                hd_name, coralie_df, hipparcos_df, hd_hip_mapping
            )
        except Exception as e:
            print(f"Error initializing {hd_name}: {str(e)}")
    return models


In [193]:
def configure_linear_params(model_dict):
    # Astrometry linear params
    model_dict['astro_model'].set_linear_params(
        x0=0.0,  # Right ascension offset
        y0=0.0,  # Declination offset
        mu_ra=0.0,  # RA proper motion
        mu_dec=0.0,  # Dec proper motion
        parallax=1e-3  # Initial parallax estimate
    )
    
    # RV linear params
    model_dict['rv_model'].set_linear_params(
        gamma={inst: 0.0 for inst in model_dict['rv_model'].instruments},
        trend=0.0,
        quad=0.0
    )


In [194]:
def optimize_linear_components(models_dict):
    results = {}
    for hd_name, model in tqdm(models_dict.items(), desc="Optimizing linear params"):
        try:
            # Fit astrometry
            model['astro_model'].fit_linear()
            
            # Fit RV
            model['rv_model'].fit_linear()
            
            # Store results
            results[hd_name] = {
                'astro_params': model['astro_model'].get_params(),
                'rv_params': model['rv_model'].get_params()
            }
        except Exception as e:
            print(f"Error optimizing {hd_name}: {str(e)}")
    return results


In [195]:
# Initialize all models
all_models = initialize_all_models(hd_hip_mapping, filtered_coralie, filtered_hipparcos)

# Configure linear parameters
for hd in all_models:
    configure_linear_params(all_models[hd])

# Optimize linear components
linear_results = optimize_linear_components(all_models)

# Diagnostic check
print(f"Successfully initialized models for {len(linear_results)}/{len(hd_hip_mapping)} stars")


Initializing models:  21%|██        | 538/2566 [00:00<00:00, 2777.55it/s]

Error initializing l5: zero-size array to reduction operation minimum which has no identity
Error initializing l2: zero-size array to reduction operation minimum which has no identity
Error initializing HD175518: zero-size array to reduction operation minimum which has no identity
Error initializing HD13445: zero-size array to reduction operation minimum which has no identity
Error initializing HD219709: zero-size array to reduction operation minimum which has no identity
Error initializing HD1926: zero-size array to reduction operation minimum which has no identity
Error initializing HD202457: zero-size array to reduction operation minimum which has no identity
Error initializing HD3222: zero-size array to reduction operation minimum which has no identity
Error initializing HD17289: zero-size array to reduction operation minimum which has no identity
Error initializing HD10647: zero-size array to reduction operation minimum which has no identity
Error initializing HD31143: zero-size a

Initializing models:  43%|████▎     | 1112/2566 [00:00<00:00, 2835.55it/s]

Error initializing 8611-00464-1: zero-size array to reduction operation minimum which has no identity
Error initializing 7174-01468-1: zero-size array to reduction operation minimum which has no identity
Error initializing 7711-01075-1: zero-size array to reduction operation minimum which has no identity
Error initializing 6629-00559-1: zero-size array to reduction operation minimum which has no identity
Error initializing 8607-00441-1: zero-size array to reduction operation minimum which has no identity
Error initializing G1005-28.528: zero-size array to reduction operation minimum which has no identity
Error initializing 8191-01982-1: zero-size array to reduction operation minimum which has no identity
Error initializing 6631-01158-1: zero-size array to reduction operation minimum which has no identity
Error initializing 7189-01892-1: zero-size array to reduction operation minimum which has no identity
Error initializing 7273-00058-1: zero-size array to reduction operation minimum wh

Initializing models:  77%|███████▋  | 1981/2566 [00:00<00:00, 2845.32it/s]

Error initializing 6391-00025-1: zero-size array to reduction operation minimum which has no identity
Error initializing 6966-00584-1: zero-size array to reduction operation minimum which has no identity
Error initializing 6389-00313-1: zero-size array to reduction operation minimum which has no identity
Error initializing 6386-01220-1: zero-size array to reduction operation minimum which has no identity
Error initializing 6389-00341-1: zero-size array to reduction operation minimum which has no identity
Error initializing G2239-05.1243: zero-size array to reduction operation minimum which has no identity
Error initializing 8009-00243-1: zero-size array to reduction operation minimum which has no identity
Error initializing 8837-00609-1: zero-size array to reduction operation minimum which has no identity
Error initializing G2317-39.42: zero-size array to reduction operation minimum which has no identity
Error initializing 6979-00264-1: zero-size array to reduction operation minimum wh

Initializing models: 100%|██████████| 2566/2566 [00:00<00:00, 2835.66it/s]


Error initializing 7465-00506-1: zero-size array to reduction operation minimum which has no identity
Error initializing 9113-00063-1: zero-size array to reduction operation minimum which has no identity
Error initializing 7976-00411-1: zero-size array to reduction operation minimum which has no identity
Error initializing 8800-01217-1: zero-size array to reduction operation minimum which has no identity
Error initializing 5187-01611-1: zero-size array to reduction operation minimum which has no identity
Error initializing 9329-00595-1: zero-size array to reduction operation minimum which has no identity
Error initializing 6345-00976-1: zero-size array to reduction operation minimum which has no identity
Error initializing 5792-00840-1: zero-size array to reduction operation minimum which has no identity
Error initializing 5796-00531-1: zero-size array to reduction operation minimum which has no identity
Error initializing 6939-00151-1: zero-size array to reduction operation minimum wh

Optimizing linear params: 0it [00:00, ?it/s]

Successfully initialized models for 0/2566 stars


In [197]:
def initialize_all_models(hd_hip_mapping, coralie_df, hipparcos_df):
    all_models = {}
    for hd, hip in tqdm(hd_hip_mapping.items(), desc="Initializing models"):
        # Filter CORALIE data
        coralie_star = coralie_df[coralie_df['obj_id_catname'] == hd]
        if coralie_star.empty:
            print(f"Skipping {hd}: No CORALIE data found")
            continue
        
        # Filter Hipparcos data (skip if HIP is None)
        if hip is None:
            print(f"Skipping {hd}: No HIP ID found")
            continue
        hipparcos_star = hipparcos_df[hipparcos_df['HIP'] == hip]
        if hipparcos_star.empty:
            print(f"Skipping {hd} (HIP {hip}): No Hipparcos data found")
            continue
        
        # Placeholder for model initialization (replace with your actual model)
        try:
            # Example: Initialize a simple dictionary; replace with kepmodel/spleaf logic
            model = {
                'coralie': coralie_star,
                'hipparcos': hipparcos_star,
                'hd': hd,
                'hip': hip
            }
            all_models[hd] = model
        except Exception as e:
            print(f"Error initializing {hd}: {e}")
    
    return all_models

def configure_linear_params(model):
    # Placeholder: Add your linear parameter configuration logic
    # For now, just pass through
    return model

def optimize_linear_components(all_models):
    # Placeholder: Add your optimization logic
    linear_results = {}
    for hd, model in all_models.items():
        try:
            # Example: Just store the model; replace with actual optimization
            linear_results[hd] = model
        except Exception as e:
            print(f"Error optimizing {hd}: {e}")
    return linear_results

# Run the pipeline
all_models = initialize_all_models(hd_hip_mapping, filtered_coralie, filtered_hipparcos)
for hd in all_models:
    configure_linear_params(all_models[hd])
linear_results = optimize_linear_components(all_models)
print(f"Successfully initialized models for {len(linear_results)}/{len(hd_hip_mapping)} stars")


Initializing models:  31%|███       | 793/2566 [00:00<00:00, 4181.25it/s]

Skipping l5: No CORALIE data found
Skipping l2: No CORALIE data found
Skipping HD175518 (HIP 92918): No Hipparcos data found
Skipping HD13445 (HIP 10138): No Hipparcos data found
Skipping HD219709 (HIP 115087): No Hipparcos data found
Skipping HD1926 (HIP 1823): No Hipparcos data found
Skipping HD202457 (HIP 105214): No Hipparcos data found
Skipping HD3222 (HIP 2743): No Hipparcos data found
Skipping HD17289 (HIP 12726): No Hipparcos data found
Skipping HD10647 (HIP 7978): No Hipparcos data found
Skipping HD31143 (HIP 22602): No Hipparcos data found
Skipping HD20916 (HIP 15539): No Hipparcos data found
Skipping HD39091 (HIP 26394): No Hipparcos data found
Skipping HD40307 (HIP 27887): No Hipparcos data found
Skipping HD30501 (HIP 22122): No Hipparcos data found
Skipping HD32778 (HIP 23437): No Hipparcos data found
Skipping HD52698 (HIP 33817): No Hipparcos data found
Skipping HD74014 (HIP 42634): No Hipparcos data found
Skipping HD197214 (HIP 102264): No Hipparcos data found
Skipping H

Initializing models:  69%|██████▊   | 1762/2566 [00:00<00:00, 3666.39it/s]

Skipping HIP9095 (HIP 9095): No Hipparcos data found
Skipping 9050-02332-1: No CORALIE data found
Skipping 8398-01998-1: No CORALIE data found
Skipping 9091-00717-1: No CORALIE data found
Skipping G2048-50.217: No CORALIE data found
Skipping 8427-00792-1: No CORALIE data found
Skipping 6436-00770-1: No CORALIE data found
Skipping 6445-00414-1: No CORALIE data found
Skipping 6442-00368-1: No CORALIE data found
Skipping 8667-00714-1: No CORALIE data found
Skipping 8747-01558-1: No CORALIE data found
Skipping 9070-00638-1: No CORALIE data found
Skipping 7960-01695-1: No CORALIE data found
Skipping 6948-00023-1: No CORALIE data found
Skipping 8004-00032-1: No CORALIE data found
Skipping 6977-01070-1: No CORALIE data found
Skipping 8846-00907-1: No CORALIE data found
Skipping 9134-01438-1: No CORALIE data found
Skipping 8338-02738-1: No CORALIE data found
Skipping 9071-01997-1: No CORALIE data found
Skipping 6957-01158-1: No CORALIE data found
Skipping 9119-01158-1: No CORALIE data found
Sk

Initializing models: 100%|██████████| 2566/2566 [00:00<00:00, 4072.83it/s]

Skipping HD30774 (HIP 22395): No Hipparcos data found
Skipping 5171-00176-1: No CORALIE data found
Skipping G2034-11.971: No CORALIE data found
Skipping G2039-16.1848: No CORALIE data found
Skipping G2039-16.1626: No CORALIE data found
Skipping 6331-01849-1: No CORALIE data found
Skipping 7465-00506-1: No CORALIE data found
Skipping 9113-00063-1: No CORALIE data found
Skipping 7976-00411-1: No CORALIE data found
Skipping 8800-01217-1: No CORALIE data found
Skipping 5187-01611-1: No CORALIE data found
Skipping 9329-00595-1: No CORALIE data found
Skipping 6345-00976-1: No CORALIE data found
Skipping 5792-00840-1: No CORALIE data found
Skipping 5796-00531-1: No CORALIE data found
Skipping 6939-00151-1: No CORALIE data found
Skipping 5801-00644-1: No CORALIE data found
Skipping 6953-00211-1: No CORALIE data found
Skipping 6954-00353-1: No CORALIE data found
Skipping 9338-00815-1: No CORALIE data found
Skipping 9344-03103-1: No CORALIE data found
Skipping 6979-00227-1: No CORALIE data found

In [203]:
filtered_hipparcos.head()

,Unnamed: 0,HIP,IORB,EPOCH,PARF,CPSI,SPSI,RES,SRES,T_BJD,S_MAS,CTH,STH,bin
0,4036262,305,305,-1.0353,0.6441,-0.3940,0.9191,-1.09,3.07,2.447971e+06,1.7341,-0.919109,-0.394004,2447970
1,4036263,305,362,-0.9660,-0.6689,0.7292,0.6843,-0.27,2.66,2.447996e+06,0.9389,-0.684300,0.729200,2447996
2,4036264,305,362,-0.9660,-0.6663,0.7267,0.6869,0.06,2.47,2.447996e+06,0.7263,-0.686926,0.726727,2447996
3,4036265,305,363,-0.9648,-0.6671,0.7251,0.6886,0.78,2.92,2.447997e+06,1.4471,-0.688621,0.725122,2447996
4,4036266,305,416,-0.9004,0.5023,-0.6959,0.7181,2.88,3.59,2.448020e+06,3.3823,-0.718120,-0.695919,2448020


In [202]:
filtered_coralie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4727 entries, 4 to 9985
Columns: 112 entries, Unnamed: 0 to th_ar2
dtypes: bool(10), float64(85), int64(1), object(16)
memory usage: 3.8+ MB


In [204]:
hd_hip_mapping

{'l5': None,
 'l2': None,
 'HD175518': 92918,
 'HD13445': 10138,
 'HD219709': 115087,
 'HD1926': 1823,
 'HD202457': 105214,
 'HD3222': 2743,
 'HD17289': 12726,
 'HD10647': 7978,
 'HD31143': 22602,
 'HD20916': 15539,
 'HD39091': 26394,
 'HD40307': 27887,
 'HD30501': 22122,
 'HD32778': 23437,
 'HD52698': 33817,
 'HD74014': 42634,
 'HD197214': 102264,
 'HD1461': 1499,
 'HD30562': 22336,
 'HD82943': 47007,
 'HD27019': 19758,
 'HD62644': 37606,
 'HD64606': 38625,
 'HD46569': 31079,
 'HD48189': 31711,
 'HIP38910': 38910,
 'HD50499': 32970,
 'HD104304': 58576,
 'HD29813': 21778,
 'HD92987': 52472,
 'HD112164': 63033,
 'HD121384': 68101,
 'HD87359': 49350,
 'HD99610': 55900,
 'HD62848': 37635,
 'HD115153': 64688,
 'HD145825': 79578,
 'HD63077': 37853,
 'HD78549': 44676,
 'HD78643': 44874,
 'HD78762': 44745,
 'HD117635': 65982,
 'HD136352': 75181,
 'HD150248': 81746,
 'HD152311': 82621,
 'HD86249': 48718,
 'HD98356': 55235,
 'HD165499': 89042,
 'HD92788': 52409,
 'HD131923': 73241,
 'HD134251':

In [209]:
from tqdm import tqdm

def initialize_all_models(hd_hip_mapping, coralie_df, hipparcos_df):
    all_models = {}
    for hd, hip in tqdm(hd_hip_mapping.items(), desc="Initializing models"):
        coralie_star = coralie_df[coralie_df['obj_id_catname'] == hd]
        if coralie_star.empty:
            print(f"Skipping {hd}: No CORALIE data found")
            continue
        
        if hip is None:
            print(f"Skipping {hd}: No HIP ID found")
            continue
        hipparcos_star = hipparcos_df[hipparcos_df['HIP'] == hip]
        if hipparcos_star.empty:
            print(f"Skipping {hd} (HIP {hip}): No Hipparcos data found")
            continue
        
        try:
            model = {
                'coralie': coralie_star,
                'hipparcos': hipparcos_star,
                'hd': hd,
                'hip': hip
            }
            all_models[hd] = model
        except Exception as e:
            print(f"Error initializing {hd}: {e}")
    
    return all_models

def configure_linear_params(model):
    return model  # Placeholder

def optimize_linear_components(all_models):
    linear_results = {}
    for hd, model in all_models.items():
        try:
            linear_results[hd] = model  # Placeholder
        except Exception as e:
            print(f"Error optimizing {hd}: {e}")
    return linear_results

# Run it
all_models = initialize_all_models(hd_hip_mapping, coralie_df, hipparcos_df)
for hd in all_models:
    configure_linear_params(all_models[hd])
linear_results = optimize_linear_components(all_models)
print(f"Successfully initialized models for {len(linear_results)}/{len(hd_hip_mapping)} stars")

Initializing models:  23%|██▎       | 58/251 [00:00<00:00, 286.75it/s]

Skipping HD175518 (HIP 92918): No Hipparcos data found
Skipping HD13445 (HIP 10138): No Hipparcos data found
Skipping HD219709 (HIP 115087): No Hipparcos data found
Skipping HD1926 (HIP 1823): No Hipparcos data found
Skipping HD202457 (HIP 105214): No Hipparcos data found
Skipping HD3222 (HIP 2743): No Hipparcos data found
Skipping HD17289 (HIP 12726): No Hipparcos data found
Skipping HD10647 (HIP 7978): No Hipparcos data found
Skipping HD31143 (HIP 22602): No Hipparcos data found
Skipping HD20916 (HIP 15539): No Hipparcos data found
Skipping HD39091 (HIP 26394): No Hipparcos data found
Skipping HD40307 (HIP 27887): No Hipparcos data found
Skipping HD30501 (HIP 22122): No Hipparcos data found
Skipping HD32778 (HIP 23437): No Hipparcos data found
Skipping HD52698 (HIP 33817): No Hipparcos data found
Skipping HD74014 (HIP 42634): No Hipparcos data found
Skipping HD197214 (HIP 102264): No Hipparcos data found
Skipping HD1461 (HIP 1499): No Hipparcos data found
Skipping HD30562 (HIP 22336)

Initializing models:  48%|████▊     | 120/251 [00:00<00:00, 297.91it/s]

Skipping HD109492 (HIP 61443): No Hipparcos data found
Skipping HD94340 (HIP 53217): No Hipparcos data found
Skipping HD119782 (HIP 67237): No Hipparcos data found
Skipping HD83443 (HIP 47202): No Hipparcos data found
Skipping HD106869 (HIP 59926): No Hipparcos data found
Skipping HD157060 (HIP 85019): No Hipparcos data found
Skipping HD181199A: No HIP ID found
Skipping HD106515A (HIP 59743): No Hipparcos data found
Skipping HD167665 (HIP 89620): No Hipparcos data found
Skipping HD154697 (HIP 83770): No Hipparcos data found
Skipping HD112863 (HIP 63419): No Hipparcos data found
Skipping HD73744 (HIP 41871): No Hipparcos data found
Skipping HD159656 (HIP 86289): No Hipparcos data found
Skipping HD102071 (HIP 57271): No Hipparcos data found
Skipping HD196067 (HIP 102125): No Hipparcos data found
Skipping HD96553 (HIP 54366): No Hipparcos data found
Skipping HD137763 (HIP 75718): No Hipparcos data found
Skipping HD111038 (HIP 62403): No Hipparcos data found
Skipping HD112575A (HIP 63257):

Initializing models:  60%|█████▉    | 150/251 [00:00<00:00, 293.18it/s]

Skipping HD192263 (HIP 99711): No Hipparcos data found
Skipping HD18907 (HIP 14086): No Hipparcos data found
Skipping HD170915 (HIP 91154): No Hipparcos data found
Skipping HD2070 (HIP 1955): No Hipparcos data found
Skipping HD11131 (HIP 8486): No Hipparcos data found
Skipping HD223078 (HIP 117247): No Hipparcos data found
Skipping HD13724 (HIP 10278): No Hipparcos data found
Skipping HD4747 (HIP 3850): No Hipparcos data found
Skipping HD5562 (HIP 4395): No Hipparcos data found
Skipping HD202206 (HIP 104903): No Hipparcos data found
Skipping HD139696 (HIP 76793): No Hipparcos data found
Skipping HD176021 (HIP 93507): No Hipparcos data found
Skipping HD3359 (HIP 2848): No Hipparcos data found
Skipping HD11264 (HIP 8498): No Hipparcos data found
Skipping HD14802 (HIP 11072): No Hipparcos data found
Skipping HD13945 (HIP 10492): No Hipparcos data found
Skipping HD18809 (HIP 14007): No Hipparcos data found
Skipping HD23576 (HIP 17483): No Hipparcos data found
Skipping HD7320 (HIP 5697): No

Initializing models:  84%|████████▍ | 212/251 [00:00<00:00, 300.85it/s]

Skipping HD59099 (HIP 36165): No Hipparcos data found
Skipping HD89707 (HIP 50671): No Hipparcos data found
Skipping HD63581 (HIP 37918): No Hipparcos data found
Skipping HD59380 (HIP 36399): No Hipparcos data found
Skipping HD118598 (HIP 66530): No Hipparcos data found
Skipping HD53680 (HIP 34052): No Hipparcos data found
Skipping HD61033 (HIP 36832): No Hipparcos data found
Skipping HD64184 (HIP 38179): No Hipparcos data found
Skipping HD161639 (HIP 87034): No Hipparcos data found
Skipping HD184860A: No HIP ID found
Skipping HD202746 (HIP 105441): No Hipparcos data found
Skipping HD212146 (HIP 110483): No Hipparcos data found
Skipping HD219175B (HIP 114703): No Hipparcos data found
Skipping HD151528 (HIP 82260): No Hipparcos data found
Skipping HD207450 (HIP 107779): No Hipparcos data found
Skipping HD196067A: No HIP ID found
Skipping HD2098 (HIP 1976): No Hipparcos data found
Skipping HD15064 (HIP 11231): No Hipparcos data found
Skipping HD14629 (HIP 10925): No Hipparcos data found


Initializing models: 100%|██████████| 251/251 [00:00<00:00, 297.01it/s]

Skipping HD221275 (HIP 116050): No Hipparcos data found
Skipping HD4392 (HIP 3578): No Hipparcos data found
Skipping HD81133 (HIP 45995): No Hipparcos data found
Skipping HD82241 (HIP 46535): No Hipparcos data found
Skipping HD30774 (HIP 22395): No Hipparcos data found
Skipping HD196554 (HIP 101852): No Hipparcos data found
Skipping HD40503 (HIP 28193): No Hipparcos data found
Skipping HD124430 (HIP 69528): No Hipparcos data found
Skipping HD201427 (HIP 104604): No Hipparcos data found
Successfully initialized models for 1/251 stars
